In [31]:
import pandas as pd
import numpy as np
import os
import json
# import dt
import re

# GM_RESULT

In [32]:
res = pd.read_csv("game_results.csv")
res.head()


,game_id,seed_game_id,players,generations,game_options,scores,createtime
0,f7cbeb6cc687,NaN,4,8,NaN,"[{""corporation"":""Poseidon"",""playerScore"":69,""p...",31/8/2020 15:05:21
1,10d3f58364b6,NaN,2,12,NaN,"[{""corporation"":""Aridor"",""playerScore"":144,""pl...",31/8/2020 15:05:21
2,ff2c9f3f3e45,NaN,4,8,NaN,"[{""corporation"":""Inventrix (breakthrough)"",""pl...",31/8/2020 15:05:21
3,cf45c3f6faa2,NaN,2,11,NaN,"[{""corporation"":""Arklight (breakthrough)"",""pla...",31/8/2020 15:05:21
4,79cef4b6a297,NaN,2,8,NaN,"[{""corporation"":""Point Luna"",""playerScore"":60,...",31/8/2020 15:05:21


In [33]:
# preprocessing
res["scores"] = res["scores"].astype(object)
res["game_options"] = res["game_options"].astype(object)
res["createtime"] = pd.to_datetime(res["createtime"], dayfirst=True)


In [34]:
# scores proprocessing
store = []
for i in res["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

In [35]:
options = []
for i in res["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

In [36]:
df2.head()

,draftVariant,corporateEra,preludeExtension,venusNextExtension,coloniesExtension,turmoilExtension,boardName,showOtherPlayersVP,customCorporationsList,customColoniesList,...,showTimers,politicalAgendasExtension,moonExpansion,requiresMoonTrackCompletion,erosCardsOption,politicalAgendasExtensionToggle,showCardsBlackList,showColoniesList,randomMAToggle,doubleCorp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df3 = pd.concat([res, df1, df2],axis=1)
df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers','politicalAgendasExtensionToggle','aresHazards','requiresVenusTrackCompletion','politicalAgendasExtensionToggle','solarPhaseOption'])

In [38]:
df4.columns
df4.to_csv('./prprocessing_data', index = False)

In [39]:
def getPlayer(df,num): # get all players name
    player_df = pd.DataFrame(columns=['player','count'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                player_df.loc[player_df.shape[0]] = [i[column]['player'].lower().strip(),1]
    ff1 = player_df.groupby(player_df["player"]).count()
    ff1["player_list"] = ff1.values.tolist()
    return ff1


In [40]:
player_count = getPlayer(df4,4)
player_count.to_csv('./pl_count.csv', index = True)

In [41]:
def playerCount(df,num,player): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' not in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [42]:
df_player = playerCount(df4,4,['ee'])
df_player.tail(20)

,id,corporation1,rank,order,score,time,generations,corporation2
0,ee,Aphrodite (breakthrough),2,1,82,2021-04-26 16:26:26,7,nan
1,ee,Manutech,1,1,74,2021-04-27 17:34:43,7,nan
2,ee,Inventrix (breakthrough),4,2,76,2021-04-28 01:41:52,8,nan
3,ee,Terralabs Research (breakthrough),1,1,85,2021-04-28 17:40:15,8,nan
4,ee,Philares,1,1,65,2021-04-29 01:01:53,5,nan
5,ee,Teractor (breakthrough),1,3,76,2021-04-30 01:36:51,7,nan
6,ee,Pharmacy Union,1,3,102,2021-05-20 00:42:47,7,nan
7,ee,Lakefront Resorts,2,2,66,2021-06-20 00:12:36,6,Mining guild (breakthrough)
8,ee,Philares,1,4,80,2021-07-03 14:13:03,6,Ecoline (breakthrough)
9,ee,Point Luna,1,3,87,2021-07-09 20:50:31,6,Teractor (breakthrough)


In [43]:
#time series
def time_series(df):
    ff1 = df[(df["players"]==4) & (df["generations"]<=9)]
    ff1.to_csv('./time_original', index = True)
    ff2 = ff1.groupby(ff1['createtime'].dt.date).mean()
    return ff2


In [44]:
# 时间绘图
df5 = time_series(df4)
df5.head(20)
df5 = df5.drop(columns=["players","initialDraftRounds"])
df5.to_csv('./time_series.csv', index = True)
# df5.plot()

# US_RESULT

In [45]:
# usage: demostrate user's creating time, like "you are the NO.1132 player"
user = pd.read_csv("users.csv")
user = user.drop(columns="prop")
user["name"] = user["name"].str.lower()
user["createtime"] = pd.to_datetime(user["createtime"], dayfirst=True)
user = user.sort_values(by=["createtime"])
user = user.reset_index(drop=True)
user["createtime"] = user.index+1
user.to_csv('./user_pre.csv', index = False)

In [136]:
# left join and mapping
user_player = user.merge(player_count,how="left",left_on="name",right_on="player")
user_player = user_player.sort_values(by=["player_list"],na_position="first")
user_player.to_csv('./user_player_mapping.csv', index = False)
user_player = user_player.set_index("name")
user_player = user_player.drop(columns="player_list")
user_player.tail()

,createtime,count
name,,
奶牛,83,154.0
阿尔,294,159.0
无语,21,176.0
钝钝,59,269.0
乙木,17,301.0


In [137]:
# detect missing user name
player_user = player_count.merge(user,how="left",left_on="player",right_on="name")
player_user = player_user.sort_values(by=["count"],ascending=False, na_position="first")
player_user = player_user[player_user["name"].isna()]
player_user.to_csv('./player_user_mapping.csv', index = False)
player_user.head()

,count,player_list,name,createtime
1443,238,[238],NaN,NaN
1302,210,[210],NaN,NaN
1444,121,[121],NaN,NaN
222,120,[120],NaN,NaN
349,106,[106],NaN,NaN


# CUSTOMIZATION

In [138]:
player_mapping_df = pd.read_csv('player_mapping.csv')
player_mapping_df = player_mapping_df.set_index("playername")

In [139]:
def playerGroup(df,num,player,map_df): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' not in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [168]:
test_df = playerGroup(df4,4,'ender',player_mapping_df)

In [175]:
group_by_corp = test_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
group_by_corp.head()

,id,rank,score,generations
corporation1,,,,
Manutech,11,1.636364,73.181818,7.272727
Lakefront Resorts,9,1.666667,73.888889,6.777778
Point Luna,9,1.666667,87.222222,7.777778
Philares,9,1.333333,72.666667,6.444444
Aphrodite (breakthrough),8,1.750000,70.500000,6.250000


In [172]:
group_by_corp = test_df.groupby(['corporation1']).count().sort_values(by=['count'], ascending=False)
favorite_corp = group_by_corp.loc[1,'corporation1']
favorite_corp_times = group_by_corp.loc[1,'count']
print(favorite_corp, favorite_corp_times)

KeyError: 'count'

In [141]:
test_df.to_csv('ee.csv',index=False)

In [183]:
def user_summary(ff1,ff2,user): ## ff1=df4
    user_df = playerGroup(ff1,4,user,player_mapping_df)
    # measure 0: check whether the account is uesable
    if pd.isna(ff2.loc[user,"createtime"].astype("int")): return []
    # measure 1: account create date
    createtime = ff2.loc[user,"createtime"]
    # measure 2: total games, favourite corp
    group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    group_by_corp = group_by_corp.reset_index()
    favorite_corp = group_by_corp.loc[0,'corporation1']
    favorite_corp_times = group_by_corp.loc[0,'id']
    games = user_df.shape[0]
    # measure 3: most frequently play time period
    # measure 4: avg score and gen
    # measure 5: highest score, with which corp
    return [user, createtime, games,favorite_corp,favorite_corp_times]

In [184]:
print(user_summary(df4,user_player,'ender'))

['ender', 2.0, <bound method DataFrame.count of         id                              corporation1 rank order score  \
0    ender                                  Poseidon    4     1    69   
1    ender                                  Philares    1     3    71   
2       e神                   Thorgate (breakthrough)    2     3    82   
3     egod                             Septem Tribus    1     2    81   
4       e神               Valley Trust (breakthrough)    2     3    53   
..     ...                                       ...  ...   ...   ...   
152     ee                                  Philares    1     4    80   
153     ee                                Point Luna    1     3    87   
154     ee  Interplanetary Cinematics (breakthrough)    1     1    85   
155     ee                         Lakefront Resorts    2     3    58   
156  ender                                     Vitor    4     4    66   

                   time generations             corporation2  
0   2020-08-